In [15]:
from torch import tensor
from torchmetrics.classification import MulticlassAccuracy
from src.trainer.metrics import Metrics

In [19]:
target = tensor([2, 2, 1, 0, 0])
preds = tensor([2, 1, 1, 0, 1])
metric = Metrics(3, 1).mca
metric.update(preds, target)
metric.compute()

tensor(0.6667)

In [12]:
metric.fn, metric.tp, metric.fp, metric.tn

(tensor([1, 0, 0]), tensor([1, 1, 1]), tensor([0, 1, 0]), tensor([2, 2, 3]))

In [13]:
(metric.tp.sum()) / (metric.tp.sum() + metric.fn.sum())

tensor(0.7500)

In [102]:
from topk.svm import SmoothTopkSVM
from torchmetrics.classification import MulticlassAccuracy

import torch
import torchvision
from torchvision.datasets import CIFAR10

In [103]:
class CustomModel(torch.nn.Module):
    def __init__(self, in_features: int, out_classes: int):
        super(CustomModel, self).__init__()
        self.linear = torch.nn.Linear(in_features, out_classes)
        
    def forward(self, x):
        return self.linear(x)

In [119]:
B = 64
C = 10
K = 3
EPOCHS = 3
alpha = 0.7
tau = 0.2

In [105]:
transformations = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [106]:
trainset = CIFAR10(root='./data', train=True, download=True, transform=transformations)
testset = CIFAR10(root='./data', train=False, download=True, transform=transformations)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=B, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=B, shuffle=False)

m1 = MulticlassAccuracy(num_classes=C, average="micro", top_k=1)
m3 = MulticlassAccuracy(num_classes=C, average="micro", top_k=K)



Files already downloaded and verified
Files already downloaded and verified


In [107]:
len(trainloader), len(testloader)

(782, 157)

In [108]:
def train(model, trainloader, criterion, optimizer, metrics):
    m1, m3 = metrics
    m1.reset()
    m3.reset()
    step = 0
    for i in range(EPOCHS):
        model.train()
        for inputs, labels in trainloader:
            outputs = model(inputs)
            
            optimizer.zero_grad()
            
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            if (step + 1) % 50 == 0:
                print(f"Step {step + 1}, loss: {loss.item()}")
            
            m1.update(outputs, labels)
            m3.update(outputs, labels)
            
            step += 1
            
        print(f"End of epoch {i+1}/{EPOCHS}, accuracies: {m1.compute().item() * 100:.2f}, {m3.compute().item() * 100:.2f}")
        
    print(f"Top-1 accuracy: {m1.compute().item() * 100:.2f}")
    print(f"Top-3 accuracy: {m3.compute().item() * 100:.2f}")

In [114]:

@torch.no_grad()
def test(model, testloader, criterion, metrics: tuple[MulticlassAccuracy, MulticlassAccuracy]):
    m1, m3 = metrics
    m1.reset()
    m3.reset()
    
    model.eval()
    loss_tot = 0
    step = 0
    for inputs, labels in testloader:
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        
        m1.update(outputs, labels)
        m3.update(outputs, labels)
        
        if (step + 1) % 50 == 0:
            print(f"Step {step + 1}, loss: {loss.item()}")
            
        loss_tot += loss.item()
        step += 1
        
    print(f"Top-1 accuracy: {m1.compute().item() * 100:.2f}")
    print(f"Top-3 accuracy: {m3.compute().item() * 100:.2f}")
    print(f"Loss: {loss_tot / len(testloader)}")

In [110]:
model = torchvision.models.mobilenet_v2(weights="DEFAULT")
model.classifier[-1] = torch.nn.Linear(model.classifier[-1].in_features, C)
for param in model.features[:15].parameters():
    param.requires_grad = False

hard_criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [111]:
train(model, trainloader, hard_criterion, optimizer, (m1, m3))

Step 50, loss: 1.6758540868759155
Step 100, loss: 1.5768574476242065
Step 150, loss: 1.6489557027816772
Step 200, loss: 1.6039656400680542
Step 250, loss: 1.3739254474639893
Step 300, loss: 1.4427231550216675
Step 350, loss: 1.4571537971496582
Step 400, loss: 1.4831770658493042
Step 450, loss: 1.4126404523849487
Step 500, loss: 1.3841454982757568
Step 550, loss: 1.558480978012085
Step 600, loss: 1.289584994316101
Step 650, loss: 1.2016512155532837
Step 700, loss: 1.3622970581054688
Step 750, loss: 1.5817837715148926
End of epoch 1/3, accuracies: 48.63, 80.10
Step 800, loss: 1.3741772174835205
Step 850, loss: 1.688345193862915
Step 900, loss: 1.4620552062988281
Step 950, loss: 1.1700478792190552
Step 1000, loss: 1.2572259902954102
Step 1050, loss: 1.2648059129714966
Step 1100, loss: 1.0765782594680786
Step 1150, loss: 1.2494066953659058
Step 1200, loss: 1.087184190750122
Step 1250, loss: 1.5420562028884888
Step 1300, loss: 1.3751832246780396
Step 1350, loss: 1.0642421245574951
Step 1400

In [115]:
test(model, testloader, hard_criterion, (m1, m3))

Step 50, loss: 1.1821616888046265
Step 100, loss: 1.1528807878494263
Step 150, loss: 1.150281310081482
Top-1 accuracy: 58.98
Top-3 accuracy: 87.01
Loss: 1.179604022366226


In [120]:
model = torchvision.models.mobilenet_v2(weights="DEFAULT")
model.classifier[-1] = torch.nn.Linear(model.classifier[-1].in_features, C)
for param in model.features[:15].parameters():
    param.requires_grad = False

smooth_criterion = SmoothTopkSVM(C, alpha, tau, K)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Setting tau to 0.2


In [121]:
train(model, trainloader, smooth_criterion, optimizer, (m1, m3))

Step 50, loss: 0.47312334179878235
Step 100, loss: 0.2778180241584778
Step 150, loss: 0.37850090861320496
Step 200, loss: 0.32945364713668823
Step 250, loss: 0.33578747510910034
Step 300, loss: 0.31762412190437317
Step 350, loss: 0.2790426015853882
Step 400, loss: 0.43221235275268555
Step 450, loss: 0.33978256583213806
Step 500, loss: 0.2622852921485901
Step 550, loss: 0.324231892824173
Step 600, loss: 0.42628371715545654
Step 650, loss: 0.28953292965888977
Step 700, loss: 0.31066590547561646
Step 750, loss: 0.5093371868133545
End of epoch 1/3, accuracies: 47.15, 80.02
Step 800, loss: 0.28892990946769714
Step 850, loss: 0.3473959267139435
Step 900, loss: 0.3689599335193634
Step 950, loss: 0.27272266149520874
Step 1000, loss: 0.24775241315364838
Step 1050, loss: 0.14146298170089722
Step 1100, loss: 0.2529749572277069
Step 1150, loss: 0.3322805166244507
Step 1200, loss: 0.24014617502689362
Step 1250, loss: 0.33436182141304016
Step 1300, loss: 0.22790366411209106
Step 1350, loss: 0.258685

In [122]:
test(model, testloader, smooth_criterion, (m1, m3))

Step 50, loss: 0.30828139185905457
Step 100, loss: 0.1869034320116043
Step 150, loss: 0.15774597227573395
Top-1 accuracy: 55.56
Top-3 accuracy: 86.16
Loss: 0.23916984719645445
